In [19]:
import numpy as np
import pandas as pd
import seaborn as sns

## Simple groupbys

In [20]:
class display(object):
    """Display HTML representation of multiple objects"""
    template = """<div style="float: left; padding: 10px;">
    <p style='font-family:"Courier New", Courier, monospace'>{0}</p>{1}
    </div>"""
    def __init__(self, *args):
        self.args = args
        
    def _repr_html_(self):
        return '\n'.join(self.template.format(a, eval(a)._repr_html_())
                         for a in self.args)
    
    def __repr__(self):
        return '\n\n'.join(a + '\n' + repr(eval(a))
                           for a in self.args)

In [21]:
df = pd.DataFrame({'key': ['A', 'B', 'C', 'A', 'B', 'C'],
                   'data': range(6)}, columns=['key', 'data'])
df

,key,data
0,A,0
1,B,1
2,C,2
3,A,3
4,B,4
5,C,5


In [22]:
df.groupby('key')

In [23]:
df.groupby('key').sum()

,data
key,
A,3
B,5
C,7


## Aggregate

We're now familiar with GroupBy aggregations with sum(), median(), and the like, but the aggregate() method allows for even more flexibility. It can take a string, a function, or a list thereof, and compute all the aggregates at once. Here is a quick example combining all these:

In [24]:
rng = np.random.RandomState(0)
df = pd.DataFrame({'key': ['A', 'B', 'C', 'A', 'B', 'C'],
                   'data1': range(6),
                   'data2': rng.randint(0, 10, 6)},
                   columns = ['key', 'data1', 'data2'])
df

,key,data1,data2
0,A,0,5
1,B,1,0
2,C,2,3
3,A,3,3
4,B,4,7
5,C,5,9


In [25]:
# with aggregate you can write one line of code
df.groupby('key').aggregate(['min',np.median,max])

data1            data2           
      min median max   min median max
key                                  
A       0    1.5   3     3    4.0   5
B       1    2.5   4     0    3.5   7
C       2    3.5   5     3    6.0   9

In [26]:
df.groupby('key').aggregate({'data1':'min',
                            'data2':'max'})

,data1,data2
key,,
A,0,5
B,1,7
C,2,9


## Filter

A filtering operation allows you to drop data based on the group properties. For example, we might want to keep all groups in which the standard deviation is larger than some critical value:

In [27]:
df

,key,data1,data2
0,A,0,5
1,B,1,0
2,C,2,3
3,A,3,3
4,B,4,7
5,C,5,9


In [28]:
df.groupby('key').std()

,data1,data2
key,,
A,2.12132,1.414214
B,2.12132,4.949747
C,2.12132,4.242641


In [29]:
def filter_function(x):
    return x['data2'].std() > 4
# this function should be true only for B and C
# and A is false
# see above.

In [30]:
display('df',"df.groupby('key').std()","df.groupby('key').filter(filter_function)")

df
  key  data1  data2
0   A      0      5
1   B      1      0
2   C      2      3
3   A      3      3
4   B      4      7
5   C      5      9

df.groupby('key').std()
       data1     data2
key                   
A    2.12132  1.414214
B    2.12132  4.949747
C    2.12132  4.242641

df.groupby('key').filter(filter_function)
  key  data1  data2
1   B      1      0
2   C      2      3
4   B      4      7
5   C      5      9

In [31]:
df.groupby('key').filter(filter_function)

,key,data1,data2
1,B,1,0
2,C,2,3
4,B,4,7
5,C,5,9


## Transform()

While aggregation must return a reduced version of the data, transformation can return some transformed version of the full data to recombine. For such a transformation, the output is the same shape as the input. A common example is to center the data by subtracting the group-wise mean:

In [32]:
df

,key,data1,data2
0,A,0,5
1,B,1,0
2,C,2,3
3,A,3,3
4,B,4,7
5,C,5,9


In [33]:
df.groupby('key').transform(lambda x:x-x.mean())

## mean
# data1 : A=1.5, B =2.5,C =3.5
# data2: A =4, B=3.5,C =6

## apply x-x.mean
# 0:data1: (0-1.5 =-1.5),data2:(5-4=1)
#1: data1:(1-2.5=-1.5),data2:(0-3.5=-3.5)

,data1,data2
0,-1.5,1.0
1,-1.5,-3.5
2,-1.5,-3.0
3,1.5,-1.0
4,1.5,3.5
5,1.5,3.0


## Apply()

In [34]:
#Suppose I want to normalize data1
def norm_by_data2(x):
    x['data1']/=x['data2'].sum()
    return x
# x['data1']=x['data1']/x['data2'].sum()
# /= because it saves me writing "x['data1']="

In [36]:
display('df',"df.groupby('key').apply(norm_by_data2)")

,key,data1,data2
0,A,0,5
1,B,1,0
2,C,2,3
3,A,3,3
4,B,4,7
5,C,5,9
,key,data1,data2
0,A,0.000000,5
1,B,0.142857,0
2,C,0.166667,3


In [37]:
# 0: data1:0/(5+3)=0
# 1: data1: 1/(0+7)=0.142857
#2: data1:2/(3+9)=2/12=0.166667

In [35]:
df.groupby('key').apply(norm_by_data2)

,key,data1,data2
0,A,0.000000,5
1,B,0.142857,0
2,C,0.166667,3
3,A,0.375000,3
4,B,0.571429,7
5,C,0.416667,9


## Create your own splits for groupby

In the simple examples presented before, we split the DataFrame on a single column name. This is just one of many options by which the groups can be defined, and we'll go through some other options for group specification here.

### A list, array, series, or index providing the grouping keys
The key can be any series or list with a length matching that of the DataFrame. For example:

In [38]:
df

,key,data1,data2
0,A,0,5
1,B,1,0
2,C,2,3
3,A,3,3
4,B,4,7
5,C,5,9


In [40]:
lis=[0,1,0,1,2,0]
display('df',"df.groupby(lis).sum()")

df
  key  data1  data2
0   A      0      5
1   B      1      0
2   C      2      3
3   A      3      3
4   B      4      7
5   C      5      9

df.groupby(lis).sum()
   data1  data2
0      7     17
1      4      3
2      4      7

In [41]:
#0: data1: 0+2+5 = 7 # see from lis where 0 have 0 you will insert those values
#0: data2: 5+3+9 = 17
#1: data1: 1+3 = 4 # in lis one is in 2nd row and fourth row 
# so from data1 take 1 &3
#1: data2: 0+3 =3
#2: data1: 4
#2: data2: 7

## Passing any function in a groupby()

In [42]:
df

,key,data1,data2
0,A,0,5
1,B,1,0
2,C,2,3
3,A,3,3
4,B,4,7
5,C,5,9


In [43]:
# I want to change key column to lowercase
df2=df.set_index('key')

In [44]:
df2

,data1,data2
key,,
A,0,5
B,1,0
C,2,3
A,3,3
B,4,7
C,5,9


In [45]:
df2.groupby(str.lower).mean()

,data1,data2
a,1.5,4.0
b,2.5,3.5
c,3.5,6.0
